# Notebook Summary

This notebook is designed to automate the process of training and evaluating ARIMA_PLUS models on Monash datasets. It calculates the Mean Absolute Scaled Error (MASE) for each model and logs the results to a Google Sheet.


1.  **Input Parameters:** Modify the `auto_arima_max_order_list_string` and `max_time_series_length_list_string` in the first code cell to specify the ARIMA orders and time series lengths you want to test.
2.  **Google Sheet Setup:** Ensure the Google Sheet specified by `google_sheet_name` and `worksheet_name` exists and that you have the necessary permissions to write to it.
3.  **Dataset Names:** The notebook uses predefined lists of dataset names (`dataset_names` and `dataset_names_v2`). If you have different datasets, update these lists accordingly.
4.  **BigQuery Project:** Make sure the `project` variable in the first code cell is set to your BigQuery project ID.
5.  **Run All:** After setting up the parameters and ensuring the Google Sheet exists, run all cells in the notebook (`Run all`). The notebook will iterate through the specified datasets, train models with different parameters, calculate MASE, and append the results to your Google Sheet.
6.  **Monitor Progress:** The notebook prints progress updates to the console, indicating which dataset and parameters are currently being processed.
7.  **Review Results:** Once the execution is complete, check your Google Sheet for the MASE scores and other details for each model.

# Set up parameters and authorize bigquery and google sheet client


In [ ]:
import gspread
from google.colab import auth
from google.cloud import bigquery
from google.auth import default as google_auth_default


# @title Input Parameters
auto_arima_max_order_list_string = "2,3,4" # @param {type:"string"}
max_time_series_length_list_string = "512,1024,2048" # @param {type:"string"}
google_sheet_name = 'MASE_score_automation' # @param {type:"string"}
worksheet_name = 'MASE_score_automation' # @param {type:"string"}
bigquery_project_id = 'bqml_monash' # @param {type:"string"}


# Convert the string inputs into lists of integers
try:
  auto_arima_max_order_list = [int(x.strip()) for x in auto_arima_max_order_list_string.split(',')]
  max_time_series_length_list = [int(x.strip()) for x in max_time_series_length_list_string.split(',')]
except ValueError as e:
   print(f"Invalid input: {e}. Please provide comma separated integers.")
else:
   print("auto_arima_max_order_list:", auto_arima_max_order_list)
   print("max_time_series_length_list:", max_time_series_length_list)



auth.authenticate_user()

# Explicitly request scopes for google_auth_default to ensure BigQuery access is prompted
credentials, _ = google_auth_default(scopes=[
    "https://www.googleapis.com/auth/spreadsheets",
    "https://www.googleapis.com/auth/drive", # Drive scope often needed for Sheets access
    "https://www.googleapis.com/auth/bigquery"
]) # Get credentials, ignore default project as we use user_defined_project
bigquery.magics.context.project = bigquery_project_id
bigquery.magics.context.credentials = credentials
from google.cloud.bigquery import magics
magics.context.credentials is credentials

# initialize gspread client
gspread_client = gspread.authorize(credentials)
bigquery_client = bigquery.Client(project=bigquery_project_id, credentials=credentials)

In [ ]:
dataset_names = ['australian_electricity_demand', 'pedestrian_counts', 'traffic_hourly', 'bitcoin',
                 'covid_deaths', 'nn5_daily', 'saugeenday', 'us_births', 'weather','nn5_weekly',
                 'solar_weekly', 'traffic_weekly','tourism_monthly','hospital', 'fred_md', 'cif_2016',
                 'tourism_quarterly', 'tourism_yearly']
dataset_names_v2 = ['electricity_hourly', 'kdd_cup_2018_without_missing', 'm4_hourly',
                    'solar_10_minutes','rideshare_without_missing', 'm4_daily',
                    'sunspot_without_missing', 'vehicle_trips_without_missing',
                    'temperature_rain_without_missing', 'dominick', 'electricity_weekly',
                    'kaggle_web_traffic_weekly', 'm4_weekly', 'car_parts_without_missing',
                    'm1_monthly', 'm3_monthly',
                    'm4_monthly','m1_quarterly', 'm3_quarterly',
                    'm4_quarterly','m3_other', 'm1_yearly', 'm3_yearly', 'm4_yearly']



name_to_freq_map = {'australian_electricity_demand':'HALF-HOURLY',
                    'pedestrian_counts':'HOURLY',
                    'traffic_hourly':'HOURLY',
                    'bitcoin':'DAILY',
                    'covid_deaths':'DAILY',
                    'nn5_daily':'DAILY',
                    'saugeenday':'DAILY',
                    'us_births':'DAILY',
                    'weather':'DAILY',
                    'nn5_weekly':'WEEKLY',
                    'solar_weekly':'WEEKLY',
                    'traffic_weekly':'WEEKLY',
                    'tourism_monthly':'MONTHLY',
                    'hospital':'MONTHLY',
                    'fred_md':'MONTHLY',
                    'cif_2016':'MONTHLY',
                    'tourism_quarterly':'QUARTERLY',
                    'tourism_yearly':'YEARLY',
                    'electricity_hourly':'HOURLY',
                    'kdd_cup_2018_without_missing':'HOURLY',
                    'm4_hourly':'HOURLY',
                    'solar_10_minutes':'TEN-MINUTE',
                    'rideshare_without_missing':'HOURLY',
                    'm4_daily':'DAILY',
                    'sunspot_without_missing':'DAILY',
                    'vehicle_trips_without_missing':'DAILY',
                    'temperature_rain_without_missing':'DAILY',
                    'dominick':'WEEKLY',
                    'electricity_weekly':'WEEKLY',
                    'kaggle_web_traffic_weekly':'WEEKLY',
                    'm4_weekly':'WEEKLY',
                    'car_parts_without_missing':'MONTHLY',
                    'm1_monthly':'MONTHLY',
                    'm3_monthly':'MONTHLY',
                    'm4_monthly':'MONTHLY',
                    'tourism_quarterly':'QUARTERLY',
                    'm1_quarterly':'QUARTERLY',
                    'm3_quarterly':'QUARTERLY',
                    'm4_quarterly':'QUARTERLY',
                    'm3_other':'QUARTERLY',
                    'm1_yearly':'YEARLY',
                    'm3_yearly':'YEARLY',
                    'm4_yearly':'YEARLY'}
freq_to_horizon_map = {
                        'TEN-MINUTE':168,
                        'HALF-HOURLY':168,
                        'HOURLY':168,
                        'DAILY':30,
                        'WEEKLY':8,
                        'MONTHLY':12,
                        'QUARTERLY':30,
                        'YEARLY':30
                      }

freq_to_m_value_map = {
                        'TEN-MINUTE':144,
                        'HALF-HOURLY':48,
                        'HOURLY':24,
                        'DAILY':7,
                        'WEEKLY':52,
                        'MONTHLY':12,
                        'QUARTERLY':4,
                        'YEARLY':1
                      }
special_horizon_map = {
                        "australian_electricity_demand":336,
                        "nn5_daily": 56,
                        "tourism_monthly": 24,
                        "m4_weekly": 13,
                        "m1_monthly": 18,
                        "m3_monthly": 18,
                        "m4_monthly":18
                      }
DATA_FREQUENCY_EMPTY = 'AUTO_FREQUENCY'
DATA_FREQUENCY_YEARLY = 'YEARLY'
DATA_FREQUENCY_QUARTERLY = 'QUARTERLY'
data_frequency_map = {
                      'TEN-MINUTE':DATA_FREQUENCY_EMPTY,
                      'HALF-HOURLY':DATA_FREQUENCY_EMPTY,
                      'HOURLY':DATA_FREQUENCY_EMPTY,
                      'DAILY':DATA_FREQUENCY_EMPTY,
                      'WEEKLY':DATA_FREQUENCY_EMPTY,
                      'MONTHLY':DATA_FREQUENCY_EMPTY,
                      'QUARTERLY':DATA_FREQUENCY_QUARTERLY,
                      'YEARLY':DATA_FREQUENCY_YEARLY
                    }

# Train Model and Evaluate

In [ ]:
import numpy as np

def calculate_mase(train_df, forecasted_df, test_df, m=1):
    errors = []

    training_sets_by_id = train_df.groupby('id').agg(
        {'value': lambda y: list(y)}
    ).to_dict('index')
    forecast_sets_by_id = forecasted_df.groupby('id').agg(
        {'value': lambda y: list(y)}
    ).to_dict('index')
    test_sets_by_id = test_df.groupby('id').agg(
        {'value': lambda y: list(y)}
    ).to_dict('index')

    for id in training_sets_by_id:
      tsts = np.array(test_sets_by_id[id]['value'])
      pts = np.array(forecast_sets_by_id[id]['value'])
      length = min(len(tsts), len(pts))
      num = np.nanmean(np.abs(tsts[0:length] - pts[0:length]))

      original_data = np.array(training_sets_by_id[id]['value'])
      interval = m
      # If history data is less than frequency, set frequency to 1
      if len(original_data) <= m:
          interval = 1
      denom = np.nanmean(np.abs(original_data[0:-interval] - original_data[interval:]))
      if np.isfinite(num / denom):
        errors.append(num / denom)
    return np.nanmean(errors)


In [ ]:
def setup_google_sheet(google_sheet_name, worksheet_name):
  """Connects to Google Sheets and prepares the worksheet."""
  print("--- Phase 2: Setup Google Sheet ---")
  try:
    spreadsheet = gspread_client.open(google_sheet_name)
    sheet = spreadsheet.worksheet(worksheet_name)
    print(
        f"Opened existing worksheet: '{worksheet_name}' in spreadsheet"
        f" '{google_sheet_name}'"
    )

    # Check if the sheet is empty to decide whether to add the header
    if not sheet.get_all_values():
      print("Worksheet is empty. Appending header row.")
      header = [
          "Dataset Name",
          "Max Time Series Length",
          "Auto ARIMA Max Order",
          "Model Creation Time",
          "Frequency",
          "Horizon",
          "M Value",
          "MASE",
          "MAE",
          "RMSE",
          "MAPE",
          "sMAPE"
      ]
      sheet.append_row(header)
    else:
      print("Worksheet already has data. Skipping header row.")

    print("--- Google Sheet setup complete. ---")
    return sheet

  except gspread.exceptions.SpreadsheetNotFound:
    print(
        f"Error: Spreadsheet '{google_sheet_name}' not found. Please"
        " create it before running the script."
    )
    return None
  except gspread.exceptions.WorksheetNotFound:
    print(
        f"Error: Worksheet '{worksheet_name}' not found in spreadsheet"
        f" '{google_sheet_name}'. Please create it before running the"
        " script."
    )
    return None

In [ ]:
import time
import pandas as pd
import numpy as np

sheet = setup_google_sheet(google_sheet_name, worksheet_name)
for dataset in dataset_names:
  for max_time_series_length in max_time_series_length_list:
    for auto_arima_max_order in auto_arima_max_order_list:
      model_name = 'bqml_tutorial.' + dataset + '_max_time_series_length' + str(max_time_series_length) + '_auto_arima_max_order' + str(auto_arima_max_order)
      train_dataset_name = 'scalar-vertex-xgcp.foundation_ts.' + dataset + '_train'
      test_dataset_name = 'scalar-vertex-xgcp.foundation_ts.' + dataset + '_test'
      data_frequency = data_frequency_map[name_to_freq_map[dataset]]

      print("Using this model name: " + model_name)
      print("Using this train dataset name: " + train_dataset_name)
      print("Using this test dataset name: " + test_dataset_name)
      print("Using this data frequency: " + str(data_frequency))

      train_query = f"""
      CREATE OR REPLACE MODEL
        `{model_name}` OPTIONS( MODEL_TYPE='ARIMA_PLUS',
          TIME_SERIES_TIMESTAMP_COL='date',
          TIME_SERIES_DATA_COL='value',
          TIME_SERIES_ID_COL='ts_id',
          AUTO_ARIMA_MAX_ORDER = {auto_arima_max_order},
          MAX_TIME_SERIES_LENGTH = {max_time_series_length},
          DATA_FREQUENCY = '{data_frequency}'
          ) AS
      SELECT
        date,
        ts_id,
        value
      FROM
        `scalar-vertex-xgcp.foundation_ts.{dataset}_train`;
      """
      start_time = time.time()
      query_job = bigquery_client.query(train_query)
      query_job.result()
      end_time = time.time()
      elapsed_time = end_time - start_time
      print(f"""Model {model_name} created successfully in {elapsed_time} seconds""")

      freq = name_to_freq_map[dataset]
      if dataset not in special_horizon_map:
        horizon = freq_to_horizon_map[freq]
      else:
        horizon = special_horizon_map[dataset]
      m = freq_to_m_value_map[freq]
      print("Using this model name: " + model_name)
      print("Using this train dataset name: " + train_dataset_name)
      print("Using this test dataset name: " + test_dataset_name)
      print("Using this freq: " + freq)
      print("Using this horizon: " + str(horizon))
      print("Using this m: " + str(m))

      forecast_query = f"""
        SELECT
          ts_id AS id,
          forecast_timestamp AS timestamp,
          forecast_value AS value
        FROM
          ML.FORECAST(MODEL `{model_name}`, STRUCT({horizon} AS horizon, 0.9 AS confidence_level)) ORDER BY id, timestamp
          """

      forecasted_df = bigquery_client.query(forecast_query).to_dataframe()
      forecasted_df['timestamp'] = pd.to_datetime(forecasted_df['timestamp'], utc=True)

      test_query = f"""
      SELECT
        ts_idd AS id,
        date AS timestamp,
        value
      FROM
        `{test_dataset_name}`
        ORDER BY id, timestamp
      """

      test_df = bigquery_client.query(test_query).to_dataframe()
      test_df['timestamp'] = pd.to_datetime(test_df['timestamp'], utc=True)

      train_query = f"""
      SELECT
        ts_id AS id,
        date AS timestamp,
        value
      FROM
        `{train_dataset_name}`
      ORDER BY id, timestamp
      """

      train_df = bigquery_client.query(train_query).to_dataframe()
      train_df['timestamp'] = pd.to_datetime(train_df['timestamp'], utc=True)

      (current_mase) = calculate_mase(train_df, forecasted_df, test_df, m)
      print("MASE: " + str(current_mase))

      evaluation_query = f"""
      SELECT
        AVG(mean_absolute_error) AS MAE,
        AVG(root_mean_squared_error) AS RMSE,
        AVG(mean_absolute_percentage_error) AS MAPE,
        AVG(symmetric_mean_absolute_percentage_error) AS sMAPE,
      FROM
      ML.EVALUATE(MODEL `{model_name}`,
        (
        SELECT ts_idd AS ts_id, date, value,
        FROM
            `{test_dataset_name}`),
        STRUCT(TRUE AS perform_aggregation, {horizon} AS horizon))
      """
      evaluation_df = bigquery_client.query(evaluation_query).to_dataframe()
      MAE = evaluation_df['MAE'][0]
      RMSE = evaluation_df['RMSE'][0]
      MAPE = evaluation_df['MAPE'][0]
      sMAPE = evaluation_df['sMAPE'][0]
      row_to_append = [
          dataset,
          max_time_series_length,
          auto_arima_max_order,
          elapsed_time,
          freq,
          horizon,
          m,
          current_mase,
          MAE,
          RMSE,
          MAPE,
          sMAPE
      ]
      sheet.append_row(row_to_append)

for dataset in dataset_names_v2:
  for max_time_series_length in max_time_series_length_list:
    for auto_arima_max_order in auto_arima_max_order_list:
      model_name = 'bqml_tutorial.' + dataset + '_max_time_series_length' + str(max_time_series_length) + '_auto_arima_max_order' + str(auto_arima_max_order)
      train_dataset_name = 'scalar-vertex-xgcp.foundation_ts.' + dataset + '_train_v2'
      test_dataset_name = 'scalar-vertex-xgcp.foundation_ts.' + dataset + '_test_v2'
      data_frequency = data_frequency_map[name_to_freq_map[dataset]]


      print("Using this model name: " + model_name)
      print("Using this train dataset name: " + train_dataset_name)
      print("Using this test dataset name: " + test_dataset_name)
      print("Using this data frequency: " + str(data_frequency))


      train_query = f"""
      CREATE OR REPLACE MODEL
        `{model_name}` OPTIONS( MODEL_TYPE='ARIMA_PLUS',
          TIME_SERIES_TIMESTAMP_COL='ds',
          TIME_SERIES_DATA_COL='y',
          TIME_SERIES_ID_COL='unique_id',
          AUTO_ARIMA_MAX_ORDER = {auto_arima_max_order},
          MAX_TIME_SERIES_LENGTH = {max_time_series_length},
          DATA_FREQUENCY = '{data_frequency}'
          ) AS
      SELECT
        ds,
        y,
        unique_id
      FROM
        `scalar-vertex-xgcp.foundation_ts.{dataset}_train_v2`;
      """
      start_time = time.time()
      query_job = bigquery_client.query(train_query)
      query_job.result()
      end_time = time.time()
      elapsed_time = end_time - start_time
      print(f"""Model {model_name} created successfully """)
      freq = name_to_freq_map[dataset]
      if dataset not in special_horizon_map:
        horizon = freq_to_horizon_map[freq]
      else:
        horizon = special_horizon_map[dataset]
      m = freq_to_m_value_map[freq]
      print("Using this model name: " + model_name)
      print("Using this train dataset name: " + train_dataset_name)
      print("Using this test dataset name: " + test_dataset_name)
      print("Using this freq: " + freq)
      print("Using this horizon: " + str(horizon))
      print("Using this m: " + str(m))

      forecast_query = f"""
        SELECT
          unique_id AS id,
          forecast_timestamp AS timestamp,
          forecast_value AS value
        FROM
          ML.FORECAST(MODEL `{model_name}`, STRUCT({horizon} AS horizon, 0.9 AS confidence_level)) ORDER BY id, timestamp
          """

      forecasted_df = bigquery_client.query(forecast_query).to_dataframe()
      forecasted_df['timestamp'] = pd.to_datetime(forecasted_df['timestamp'], utc=True)

      test_query = f"""
      SELECT
        ds AS timestamp,
        y as value,
        unique_id as id
      FROM
        `{test_dataset_name}` ORDER BY id, timestamp
      """

      test_df = bigquery_client.query(test_query).to_dataframe()
      test_df['timestamp'] = pd.to_datetime(test_df['timestamp'], utc=True)

      train_query = f"""
      SELECT
        ds as timestamp,
        y as value,
        unique_id as id
      FROM
        `{train_dataset_name}` ORDER BY id, timestamp
      """

      train_df = bigquery_client.query(train_query).to_dataframe()
      train_df['timestamp'] = pd.to_datetime(train_df['timestamp'], utc=True)

      (current_mase) = calculate_mase(train_df, forecasted_df, test_df, m)
      print("MASE: " + str(current_mase))

      evaluation_query = f"""
      SELECT
        AVG(mean_absolute_error) AS MAE,
        AVG(root_mean_squared_error) AS RMSE,
        AVG(mean_absolute_percentage_error) AS MAPE,
        AVG(symmetric_mean_absolute_percentage_error) AS sMAPE,
      FROM
      ML.EVALUATE(MODEL `{model_name}`,
        (
        SELECT unique_id, ds, y,
        FROM
            `{test_dataset_name}`),
        STRUCT(TRUE AS perform_aggregation, {horizon} AS horizon))
      """
      evaluation_df = bigquery_client.query(evaluation_query).to_dataframe()
      MAE = evaluation_df['MAE'][0]
      RMSE = evaluation_df['RMSE'][0]
      MAPE = evaluation_df['MAPE'][0]
      sMAPE = evaluation_df['sMAPE'][0]
      row_to_append = [
          dataset,
          max_time_series_length,
          auto_arima_max_order,
          elapsed_time,
          freq,
          horizon,
          m,
          current_mase,
          MAE,
          RMSE,
          MAPE,
          sMAPE
      ]
      sheet.append_row(row_to_append)

